In [1]:
##

In [1]:
import pandas as pd 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [8]:
##pip install

In [9]:
pip install nltk pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [38]:
# Download the set of stopwords from NLTK

In [55]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
## import data 

In [10]:
df = pd.read_csv('Emotions_training.csv') 

Index(['text', 'label'], dtype='object')


In [11]:
df.head(10)

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
5,ive been feeling a little burdened lately wasn...,0
6,ive been taking or milligrams or times recomme...,5
7,i feel as confused about life as a teenager or...,4
8,i have been with petronas for years i feel tha...,1
9,i feel romantic too,2


In [ ]:
## check duplicates

In [12]:
df.drop_duplicates()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [ ]:
## about data 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [8]:
## remove the upper case in text data

In [14]:
entirely_uppercase_names = df[df['text'].str.isupper()]

In [15]:
print(entirely_uppercase_names)

Empty DataFrame
Columns: [text, label]
Index: []


In [11]:
## check null value in data set 

In [16]:
null_name_rows = df[df['text'].isnull()]
print(null_name_rows)

Empty DataFrame
Columns: [text, label]
Index: []


In [13]:
## remove next line  in the data set 

In [17]:
df['text'] = df['text'].str.replace('\n', ' ', regex=False)

In [18]:
print(df.head())

                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3


In [19]:
## Remove the link in data set

In [19]:
url_pattern = r'https?://\S+|www\.\S+'

In [20]:
df['text'] = df['text'].str.replace(url_pattern, '', regex=True)

In [21]:
print(df.head(10))

                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3
5  ive been feeling a little burdened lately wasn...      0
6  ive been taking or milligrams or times recomme...      5
7  i feel as confused about life as a teenager or...      4
8  i have been with petronas for years i feel tha...      1
9                                i feel romantic too      2


In [25]:
##Remove extra spaces between words

In [23]:
df['text'] = df['text'].str.strip()  
df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)

In [24]:
print(df.head(10))

                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3
5  ive been feeling a little burdened lately wasn...      0
6  ive been taking or milligrams or times recomme...      5
7  i feel as confused about life as a teenager or...      4
8  i have been with petronas for years i feel tha...      1
9                                i feel romantic too      2


In [29]:
##  find the special characters in a text column

In [25]:
special_char_pattern = r'[^a-zA-Z0-9\s,\.!?;:]'

In [32]:
df['has_special_char'] = df['text'].str.contains(special_char_pattern, regex=True)

In [33]:
df_special_chars = df[df['has_special_char'] == True]

In [34]:
print(df_special_chars)

Empty DataFrame
Columns: [text, label, has_special_char, text_column_no_stopwords]
Index: []


In [40]:
##Removing stopwords from a text column

In [35]:
def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(filtered_words)
df['text'] = df['text'].astype(str)
df['text_column_no_stopwords'] = df['text'].apply(remove_stopwords)

In [36]:
print(df[['text', 'text_column_no_stopwords']].head(10))

                                                text  \
0                            i didnt feel humiliated   
1  i can go from feeling so hopeless to so damned...   
2   im grabbing a minute to post i feel greedy wrong   
3  i am ever feeling nostalgic about the fireplac...   
4                               i am feeling grouchy   
5  ive been feeling a little burdened lately wasn...   
6  ive been taking or milligrams or times recomme...   
7  i feel as confused about life as a teenager or...   
8  i have been with petronas for years i feel tha...   
9                                i feel romantic too   

                            text_column_no_stopwords  
0                              didnt feel humiliated  
1  go feeling hopeless damned hopeful around some...  
2          im grabbing minute post feel greedy wrong  
3  ever feeling nostalgic fireplace know still pr...  
4                                    feeling grouchy  
5      ive feeling little burdened lately wasnt sure 

In [37]:
stop_words = set(stopwords.words('english'))


In [ ]:
## pre-process and cleaning 

In [39]:
def remove_stopwords_optimized(text, stop_words):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [40]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    text = text.lower()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vikash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
df['cleaned_text'] = df['text'].apply(clean_text)

In [42]:
df['text_column_no_stopwords'] = df['text'].apply(lambda x: remove_stopwords_optimized(x, stop_words))

In [ ]:
## making department  of the analysis 

In [43]:
df['tokens'] = df['cleaned_text'].apply(lambda x: x.split())

In [44]:
df['text_length'] = df['cleaned_text'].apply(len)
df['word_count'] = df['tokens'].apply(len)

In [53]:
df.head(100)

,text,label,has_special_char,text_column_no_stopwords,cleaned_text,tokens
0,i didnt feel humiliated,0,False,didnt feel humiliated,didnt feel humiliated,"[didnt, feel, humiliated]"
1,i can go from feeling so hopeless to so damned...,0,False,go feeling hopeless damned hopeful around some...,go feeling hopeless damned hopeful around some...,"[go, feeling, hopeless, damned, hopeful, aroun..."
2,im grabbing a minute to post i feel greedy wrong,3,False,im grabbing minute post feel greedy wrong,im grabbing minute post feel greedy wrong,"[im, grabbing, minute, post, feel, greedy, wrong]"
3,i am ever feeling nostalgic about the fireplac...,2,False,ever feeling nostalgic fireplace know still pr...,ever feeling nostalgic fireplace know still pr...,"[ever, feeling, nostalgic, fireplace, know, st..."
4,i am feeling grouchy,3,False,feeling grouchy,feeling grouchy,"[feeling, grouchy]"
...,...,...,...,...,...,...
95,i feel like throwing away the shitty piece of ...,0,False,feel like throwing away shitty piece shit paper,feel like throwing away shitty piece shit paper,"[feel, like, throwing, away, shitty, piece, sh..."
96,im starting to feel wryly amused at the banal ...,1,False,im starting feel wryly amused banal comedy err...,im starting feel wryly amused banal comedy err...,"[im, starting, feel, wryly, amused, banal, com..."
97,i find every body beautiful and only want peop...,1,False,find every body beautiful want people feel vit...,find every body beautiful want people feel vit...,"[find, every, body, beautiful, want, people, f..."
98,i hear are owners who feel victimized by their...,0,False,hear owners feel victimized associations assoc...,hear owner feel victimized association associa...,"[hear, owner, feel, victimized, association, a..."


In [ ]:
## lemmatization on the text column 

In [45]:
lemmatizer = WordNetLemmatizer()

In [47]:
lemmatizer = WordNetLemmatizer()

# Function to convert nltk tag to wordnet tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Function to lemmatize text
def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]
    return ' '.join(lemmatized_words)

# Example usage
df['lemmatized_text'] = df['text'].apply(lemmatize_text)

In [ ]:
all pre -process 

In [48]:
df['text'] = df['text'].astype(str)
df['lemmatized_text'] = df['text'].apply(lemmatize_text)

In [49]:
print(df[['text', 'lemmatized_text']].head())

                                                text  \
0                            i didnt feel humiliated   
1  i can go from feeling so hopeless to so damned...   
2   im grabbing a minute to post i feel greedy wrong   
3  i am ever feeling nostalgic about the fireplac...   
4                               i am feeling grouchy   

                                     lemmatized_text  
0                             i didnt feel humiliate  
1  i can go from feel so hopeless to so damn hope...  
2       im grab a minute to post i feel greedy wrong  
3  i be ever feel nostalgic about the fireplace i...  
4                                  i be feel grouchy  


In [ ]:
##  after complete the all data pre-process final data download  

In [50]:
df.to_csv('preprocessed_Emotions_training.csv', index=False)